In [13]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import pandas as pd
import numpy as np
from functions import get_data, make_model
import pickle
import datetime
from datetime import datetime, timedelta
import oandapyV20.endpoints.instruments as instruments
client = oandapyV20.API(access_token='199d815eb1f94c176a86dd3d5cc991a5-5e6ff7a15c524bf65fc0af5ae9a11972')

In [3]:
instrument = "EUR_USD"
params = {
    "from": "2020-01-20T00:00:00Z",
    "to": "2020-10-31T00:00:00Z",
    "granularity": "D",
    "count": 5000,
}
directory = 'training/v3/_1/'

get_data(instrument, params, "csv/daily_candles.csv", client)

In [3]:
params['from'] = "2020-10-23T16:00:00Z"
params['granularity'] = 'M15'
get_data(instrument, params, 'csv/five_min_candles.csv', client)

In [4]:
def cci(df, period=20):
    df['TP'] = (df['high']+df['low']+df['close'])/3
    df['TP_SMA']=0.0
    df['CCI']=0.0
    for i in range(period, len(df)):
        df.loc[i, 'TP_SMA'] = np.mean(df.loc[i-period:i, 'TP'])
        mean_deviation = np.mean(np.abs(df.loc[i, 'TP_SMA'] - df.loc[i-period:i, 'TP']))
        df.loc[i, 'CCI'] = (df.loc[i, 'TP'] - df.loc[i, 'TP_SMA'])/(1.5 * mean_deviation)

def stochastic(df, period=14): # same as %K
    df['%K'] = 0.0
    for i in range(period, len(df)):
        high = np.max(df.loc[i-period:i, 'high'])
        low = np.min(df.loc[i-period:i, 'low'])
        close = df.loc[i, 'close']
        df.loc[i, '%K'] = (close-low)/(high-low)
def rsi(df, period=14):
    df['RSI'], df['mean_gain'], df['mean_loss'] = 0.0, 0.0, 0.0
    gain, loss = 0, 0
    for j in range(0, period):
        change = df.loc[j+1, 'close']/df.loc[j, 'close']
        if change>=1:
            gain += change-1
        else:
            loss += 1-change
    df.loc[period, 'mean_gain'] = gain/period
    df.loc[period, 'mean_loss'] = loss/period
    RS = gain/loss
    df.loc[period, 'RSI'] = 1 - 1/(1+RS)
    for i in range(period+1, len(df)):
        gain, loss = 0, 0
        change = df.loc[i, 'close']/df.loc[i-1, 'close']
        if change>=1:
            gain += change-1
        else:
            loss += 1-change
        df.loc[i, 'mean_gain'] = (df.loc[i-1, 'mean_gain']*13 + gain)/period
        df.loc[i, 'mean_loss'] = (df.loc[i-1, 'mean_loss']*13 + loss)/period
        RS = df.loc[i, 'mean_gain']/df.loc[i, 'mean_loss']
        df.loc[i, 'RSI'] = 1 - 1/(1+RS)
def ema(df, period):
    s = 'ema-{}'.format(period)
    df[s]=0.0
    mean = np.mean(df.loc[:period, 'close'])
    multiplier = 2/(1+period)
    df.loc[period, s] = df.loc[period, 'close']*multiplier + mean*(1-multiplier)
    for i in range(period+1, len(df)):
        df.loc[i, s] = df.loc[i, 'close']*multiplier + df.loc[i-1, s]*(1-multiplier)

def atr(df, period=6):
    df['ATR']=0.0
    df['TR']=0.0
    for i in range(1, len(df)):
        h, l, c = df.loc[i, 'high'], df.loc[i, 'low'], df.loc[i-1, 'close']
        df.loc[i, 'TR'] = np.max([h-l, abs(h-c), abs(l-c)])
    for i in range(period, len(df)):
        df.loc[i, 'ATR'] = np.mean(df.loc[i-period+1:i+1, 'TR'])
def add_indicators(df):
    cci(df)
    stochastic(df)
    rsi(df)
    ema(df, 5)
    ema(df, 10)
    ema(df, 20)
    atr(df)

In [31]:
daily = pd.read_csv("csv/daily_candles.csv")
fifteen = pd.read_csv("csv/five_min_candles.csv")
daily[['open', 'high', 'low', 'close', 'volume', 'range', 'change']] = daily[['open', 'high', 'low', 'close', 'volume', 'range', 'change']].astype(np.float64)
fifteen[['open', 'high', 'low', 'close', 'volume', 'range', 'change']] = fifteen[['open', 'high', 'low', 'close', 'volume', 'range', 'change']].astype(np.float64)

add_indicators(daily), add_indicators(fifteen)
daily = daily[20:200]
future = fifteen[308: 348]
future = future.reset_index(drop=True)
future = future[['open', 'high', 'low', 'close', 'volume', 'range', 'change']]
fifteen = fifteen[20:308]

In [29]:
future.tail()

,time,open,high,low,close,volume,range,change
35,2020-10-29T05:45:00,1.17518,1.17551,1.17514,1.17550,422.0,0.00037,0.00032
36,2020-10-29T06:00:00,1.17548,1.17552,1.17509,1.17534,711.0,0.00043,-0.00014
37,2020-10-29T06:15:00,1.17533,1.17572,1.17529,1.17564,618.0,0.00043,0.00031
38,2020-10-29T06:30:00,1.17562,1.17576,1.17538,1.17540,492.0,0.00038,-0.00022
39,2020-10-29T06:45:00,1.17540,1.17571,1.17540,1.17548,476.0,0.00031,0.00008


In [14]:
import json
params = {
    "from": "2021-02-12T14:30:00Z",
    "to": "2021-02-12T14:35:00Z",
    "granularity": "M15",
    'smooth': "True"
}
r = instruments.InstrumentsCandles(instrument, params)
response = client.request(r)
c = [response['candles'][i]['mid'] for i in range(len(response['candles']))]
print(c)

params['to'] = "2021-02-12T14:40:00Z"
r = instruments.InstrumentsCandles(instrument, params)
response = client.request(r)
c = [response['candles'][i]['mid'] for i in range(len(response['candles']))]
print(c)

[{'o': '1.20892', 'h': '1.20938', 'l': '1.20856', 'c': '1.20935'}]
[{'o': '1.20892', 'h': '1.20938', 'l': '1.20856', 'c': '1.20935'}]


In [30]:
x, y = [], []
format = "%Y-%m-%dT%H:%M:%SZ"
loss, gain = 10, 30
price_scaling = ['open', 'high', 'low', 'close', 'TP', 'TP_SMA', 'ema-5', 'ema-10', 'ema-20']
thousand_scaling = ['range', 'change', 'ATR']
training_features = ['open', 'high', 'low', 'close', 'volume', 'range', 'change', 'TP', 'TP_SMA', 'CCI', '%K', 'RSI', 'ema-5', 'ema-10', 'ema-20', 'ATR']
time = datetime(year=2020, month=10, day=28, hour=21, minute=0, second=0)
time2 = datetime(year=2020, month=10, day=29, hour=6, minute=45, second=0)
mins = 15
min_delta = timedelta(minutes=1)
fif_delta = timedelta(minutes=15)
twoday_delta = timedelta(days=2)

In [ ]:
def market_closed(t):
    if (t.weekday()==4 and t.hour>=21) or (t.weekday()==5) or (t.weekday()==6 and t.hour<22):
        t = t+twoday_delta
def data_batch(params):
    r = instruments.InstrumentsCandles(instrument, params)
    response = client.request(r)
    new = []
    for i in range(len(response['candles'])):
        c = response['candles'][i]['mid']
        new.append(
            {'open': float(c['o']),
           'high': float(c['h']),
           'low': float(c['l']),
           'close': float(c['c']),
           'range': float(c['h'])-float(c['l']),
           'change': float(c['c'])-float(c['o'])})
    return new

def update_data():
    global time2, mins, future, time
    if mins==15:
        fifteen.drop(fifteen.index[0], inplace=True)
        mins=0
        while True:
            mins += 1
            params = {
                "from": time.strftime(format),
                "count": (time+min_delta).strftime(format),
                "granularity": "M1",
            }
            new = data_batch(params)
            time = time+min_delta
            market_closed(time)
            if len(new)>0:
                fifteen.append(new[0], ignore_index=True)
                break
        # update time2
        time2 = time2+fif_delta
        market_closed(time2)

        # add new 15 min candle to end of future
        params = {
            "from": time2.strftime(format),
            "count": 1,
            "granularity": "M15",
        }
        new = data_batch(params)
        future.append(new[0], ignore_index=True)

        # remove first 15 min candle and replace with about 14 minute candles
        future.drop(future.index[0], inplace=True)
        params = {
            "from": (time+min_delta).strftime(format),
            "count": 15-mins,
            "granularity": "M1",
        }
        new = data_batch(params)
        df = pd.DataFrame()
        for i in new:
            df.append(i, ignore_index=True)
        future = pd.concat((df, future), axis=0, ignore_index=True)
    else:



In [ ]:
def add_training_data():
    d, f = daily[training_features].copy(), fifteen[training_features].copy()
    d[price_scaling] = (d[price_scaling]-1.15)*5
    f[price_scaling] = (f[price_scaling]-1.15)*5
    f[thousand_scaling] *= 10e3
    d[thousand_scaling] *= 10e3
    d['range', 'change'] /= 10
    d['volume'] /= 10e5
    f['volume'] /= 10e3
    x.append(np.concatenate((d, f), axis=0))
    close = f.loc[-1, 'close']
    up = True
    down = True
    added = False
    for j in range(len(future)):
        if down and future.loc[j, 'high']-close>=0.0001*loss*5:
            down = False
        elif up and future.loc[j, 'low']-close<=-0.0001*loss*5:
            up = False
        if future.loc[j, 'high']-close>=0.0001*gain*5 and up:
            y.append(np.array([0, 0, 1]))
            added = True
            break
        if future.loc[j, 'low']-close<=-0.0001*gain*5 and down:
            y.append(np.array([1, 0, 0]))
            added = True
            break
        if not down and not up:
            y.append(np.array([0, 1, 0]))
            added = True
            break
    if not added:
        y.append(np.array([0, 1, 0]))